# Master course in Object Recognition
## Practice 1

### Title: Deep learning advanced architectures

The goal is to practice advanced deep learning architectures for multi-label classification in [Pascal VOC dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html). We specifically check ResNet50, Inception and MobileNet. We will see 1) how pretrained ResNet50 on imagenet performs on multi-label images, 2) how to modify classification head and 3) implementation of F1 metric.

### NOTES

- Hyperparameters are modifiable,
- The dataset is PASCAL VOC 2012,
- The code uses the KERAS library,
- The code can run in google colab.
- How to finetune on a pretrained model not included (i.e. freeze the pretrained network and train the head, then finetune everything),
- No validation set has been defined. The test and validation sets are the same.

In [5]:
# Imports

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from keras.layers import (
    Dense,
    GlobalAveragePooling2D,
)
import tensorflow.keras.applications as app
import pandas as pd
import csv
import time

In [10]:
# Importing from .py files

from metrics import f1_metric, mean_average_precision
from config import *
from experiment_config import experiments
from train_and_test import train_and_test
from load_data import load_data, create_dataset

In [7]:
train_list = load_data(TRAIN_TXT)
test_list = load_data(TEST_TXT)

# Create dictionaries to store datasets for different batch sizes
train_datasets = {}
test_datasets = {}

start_time = time.time()
# Iterate over batch sizes and create datasets
for batch_size in BATCH_SIZES:
    train_datasets[batch_size] = create_dataset(
        train_list, batch_size, is_training=True
    )
    test_datasets[batch_size] = create_dataset(test_list, batch_size, is_training=False)
print(f"Time taken to create datasets: {time.time() - start_time} seconds")

Time taken to create datasets: 0.9064459800720215 seconds


In [8]:
# Run model experiments
exp_name = "model-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp.warm_up

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp.learning_rate * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp.learning_rate)

    # Compile model
    model.compile(
        loss=exp.loss,
        optimizer=warmup_optimizer if exp.warm_up else opt_rms,
        metrics=["AUC", f1_metric, mean_average_precision],
    )

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: resnet50 no-pretraining no-warmup
In training loop: resnet50 no-pretraining no-warmup
Time taken for training one epoch: 42.08s
Epoch 0 training loss: 0.47, acc: 0.63, f1: 0.27, mAP: 0.45
Time taken for testing one epoch: 7.38s
Epoch 0 test loss: 0.49, acc: 0.71, f1: 0.45, mAP: 0.53
Time taken for training one epoch: 31.07s
Epoch 1 training loss: 0.19, acc: 0.86, f1: 0.46, mAP: 0.71
Time taken for testing one epoch: 6.27s
Epoch 1 test loss: 0.33, acc: 0.63, f1: 0.01, mAP: 0.24
Time taken for training one epoch: 31.52s
Epoch 2 training loss: 0.12, acc: 0.94, f1: 0.59, mAP: 0.79
Time taken for testing one epoch: 6.12s
Epoch 2 test loss: 0.55, acc: 0.46, f1: 0.02, mAP: 0.16
Training (resnet50 no-pretraining no-warmup) finished in: 124.46 seconds
Results saved to /Users/sheena/Development/master-dev/semester-2/mai-object-recognition/practicals/p1/data/02_results/model-experiments.csv
Model saved to /Users/sheena/Development/master-dev/semester-2/mai-object-recognition/pract

In [9]:
# Determine the best experiment of the 9 model experiments

df = pd.read_csv(RESULTS_DIR / f"model-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_model_experiment_config = next(
    exp for exp in experiments["model-experiments"] if exp.id == best_id
)

best_model_experiment_config

ExperimentConfig(id=0, title='resnet50 no-pretraining no-warmup', net_name=['resnet50', 'ResNet50'], train_from_scratch=True, warm_up=False, batch_size=32, n_epochs=3, last_layer_activation='sigmoid', learning_rate=0.001, loss='binary_crossentropy')

In [11]:
# Run hyperparameter experiments

exp_name = "hyperparameter-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_model_experiment_config.net_name}, train_from_scratch: {best_model_experiment_config.train_from_scratch}, warm_up: {best_model_experiment_config.warm_up} from best model experiment"
    )
    exp.net_name = best_model_experiment_config.net_name
    exp.train_from_scratch = best_model_experiment_config.train_from_scratch
    exp.warm_up = best_model_experiment_config.warm_up

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp.warm_up

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp.learning_rate * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp.learning_rate)

    # Compile model
    model.compile(
        loss=exp.loss,
        optimizer=warmup_optimizer if exp.warm_up else opt_rms,
        metrics=["AUC", f1_metric, mean_average_precision],
    )

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: batch_size: 16, learning_rate: 0.001
Reusing net_name: ['resnet50', 'ResNet50'], train_from_scratch: True, warm_up: False from best model experiment
In training loop: batch_size: 16, learning_rate: 0.001
Time taken for training one epoch: 24.91s
Epoch 0 training loss: 0.52, acc: 0.59, f1: 0.23, mAP: 0.38
Time taken for testing one epoch: 5.01s
Epoch 0 test loss: 0.42, acc: 0.63, f1: 0.29, mAP: 0.47
Time taken for training one epoch: 17.22s
Epoch 1 training loss: 0.25, acc: 0.77, f1: 0.34, mAP: 0.55
Time taken for testing one epoch: 3.65s
Epoch 1 test loss: 0.34, acc: 0.62, f1: 0.00, mAP: 0.25
Time taken for training one epoch: 15.54s
Epoch 2 training loss: 0.16, acc: 0.91, f1: 0.49, mAP: 0.73
Time taken for testing one epoch: 3.82s
Epoch 2 test loss: 0.32, acc: 0.65, f1: 0.07, mAP: 0.33
Training (batch_size: 16, learning_rate: 0.001) finished in: 70.16 seconds
Results saved to /Users/sheena/Development/master-dev/semester-2/mai-object-recognition/practicals/p1/data/02_r

KeyboardInterrupt: 

In [ ]:
# Determine the best experiment of the 9 hyperparameter experiments

df = pd.read_csv(RESULTS_DIR / f"hyperparameter-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_hyperparameter_experiment_config = next(
    exp for exp in experiments["hyperparameter-experiments"] if exp.id == best_id
)

best_hyperparameter_experiment_config

In [ ]:
# Run augmentation experiments

exp_name = "augmentation-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_hyperparameter_experiment_config.net_name}, train_from_scratch: {best_hyperparameter_experiment_config.train_from_scratch}, warm_up: {best_hyperparameter_experiment_config.warm_up} from best hyperparameter experiment"
    )
    exp.net_name = best_hyperparameter_experiment_config.net_name
    exp.train_from_scratch = best_hyperparameter_experiment_config.train_from_scratch
    exp.warm_up = best_hyperparameter_experiment_config.warm_up
    exp.batch_size = best_hyperparameter_experiment_config.batch_size
    exp.learning_rate = best_hyperparameter_experiment_config.learning_rate
    exp.loss = best_hyperparameter_experiment_config.loss
    exp.last_layer_activation = (
        best_hyperparameter_experiment_config.last_layer_activation
    )

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp.warm_up

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp.learning_rate * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp.learning_rate)

    # Compile model
    model.compile(
        loss=exp.loss,
        optimizer=warmup_optimizer if exp.warm_up else opt_rms,
        metrics=["AUC", f1_metric, mean_average_precision],
    )

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

In [ ]:
# Run classifier head experiments

exp_name = "classfier_head_experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_hyperparameter_experiment_config.net_name}, train_from_scratch: {best_hyperparameter_experiment_config.train_from_scratch}, warm_up: {best_hyperparameter_experiment_config.warm_up} from best hyperparameter experiment"
    )
    exp.net_name = best_hyperparameter_experiment_config.net_name
    exp.train_from_scratch = best_hyperparameter_experiment_config.train_from_scratch
    exp.warm_up = best_hyperparameter_experiment_config.warm_up
    exp.batch_size = best_hyperparameter_experiment_config.batch_size
    exp.learning_rate = best_hyperparameter_experiment_config.learning_rate
    exp.loss = best_hyperparameter_experiment_config.loss
    exp.last_layer_activation = (
        best_hyperparameter_experiment_config.last_layer_activation
    )

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Apply warm-up strategy
    base_model.trainable = not exp.warm_up

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Define optimizers
    warmup_optimizer = optimizers.RMSprop(learning_rate=exp.learning_rate * 0.1)
    opt_rms = optimizers.RMSprop(learning_rate=exp.learning_rate)

    # Compile model
    model.compile(
        loss=exp.loss,
        optimizer=warmup_optimizer if exp.warm_up else opt_rms,
        metrics=["AUC", f1_metric, mean_average_precision],
    )

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )